# 本模块实现特征衍生

In [2]:
import pandas as pd
import numpy as np
dat = pd.read_csv('data/train.tsv', sep='\t',index_col = 'PERSONID')
dat_testA = pd.read_csv('data/test_A.tsv', sep = '\t',index_col = 'PERSONID')

In [3]:
data = pd.concat([dat,dat_testA], axis = 0)

In [4]:
data.shape

(1600648, 54)

In [11]:
dat.columns

Index(['APPLYNO', 'FTR0', 'FTR1', 'FTR2', 'FTR3', 'FTR4', 'FTR5', 'FTR6',
       'FTR7', 'FTR8', 'FTR9', 'FTR10', 'FTR11', 'FTR12', 'FTR13', 'FTR14',
       'FTR15', 'FTR16', 'FTR17', 'FTR18', 'FTR19', 'FTR20', 'FTR21', 'FTR22',
       'FTR23', 'FTR24', 'FTR25', 'FTR26', 'FTR27', 'FTR28', 'FTR29', 'FTR30',
       'FTR31', 'FTR32', 'FTR33', 'FTR34', 'FTR35', 'FTR36', 'FTR37', 'FTR38',
       'FTR39', 'FTR40', 'FTR41', 'FTR42', 'FTR43', 'FTR44', 'FTR45', 'FTR46',
       'FTR47', 'FTR48', 'FTR49', 'FTR50', 'FTR51', 'CREATETIME'],
      dtype='object')

In [9]:
type(data)

pandas.core.frame.DataFrame

In [7]:
dat.head(30)

,APPLYNO,FTR0,FTR1,FTR2,FTR3,FTR4,FTR5,FTR6,FTR7,FTR8,...,FTR43,FTR44,FTR45,FTR46,FTR47,FTR48,FTR49,FTR50,FTR51,CREATETIME
PERSONID,,,,,,,,,,,,,,,,,,,,,
8ba94305557a4ec72435a9eb78001350,20fab21284e57a4ccd3127553920e37b,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000832,0.050000,0.0,0.0,0.000000,0.050012,0.0,0.001532,A2B186C398E0D0,2015-09-14
8ba94305557a4ec72435a9eb78001350,73f32b8b6e199129abb8fe0cb6d6c867,0.033333,0.0,0.0,0.0,0.0,0.007829,0.0,0.0,0.000000,...,0.002801,0.033333,0.0,0.0,0.000000,0.033333,0.0,0.005156,A24B176C1239E0D0,2015-09-14
8ba94305557a4ec72435a9eb78001350,324da7a0b1a5c1d3b321685d734a9dc8,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.050000,0.0,0.0,0.000023,0.050000,0.0,0.000000,A0B0C6E0D0,2015-09-14
8ba94305557a4ec72435a9eb78001350,341ceb4ae7b0ca1c46653f5a116d4bd2,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.050000,0.0,0.0,0.000023,0.050000,0.0,0.000000,A0B0C6E0D0,2015-10-16
8ba94305557a4ec72435a9eb78001350,ca260c60e62de319e38a475e2455afb8,0.050000,0.0,0.0,0.0,0.0,0.007829,0.0,0.0,0.000000,...,0.002801,0.050000,0.0,0.0,0.000000,0.050000,0.0,0.005156,A24B176C1239E0D0,2015-10-16
8ba94305557a4ec72435a9eb78001350,55eac334641dd47c43d5f95c5e2b6e6f,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000832,0.050000,0.0,0.0,0.000000,0.050000,0.0,0.001532,A2B186C398E0D0,2015-10-16
8ba94305557a4ec72435a9eb78001350,ce6cf113cde7df53b776dafb77049487,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.050000,0.0,0.0,0.000023,0.050000,0.0,0.000000,A0B0C6E0D0,2015-11-17
8ba94305557a4ec72435a9eb78001350,f699af5bb8589446c7a1d06632b35f7b,0.050000,0.0,0.0,0.0,0.0,0.007829,0.0,0.0,0.000000,...,0.002801,0.050000,0.0,0.0,0.000000,0.050000,0.0,0.005156,A24B3634C1239E0D0,2015-11-17
8ba94305557a4ec72435a9eb78001350,b55e5b0bd3817ffb897e1dcaf430f3ca,0.050000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000832,0.050000,0.0,0.0,0.000000,0.050000,0.0,0.001532,A2B287C398E0D0,2015-11-17


In [12]:
dat_id = pd.read_csv('data/train_id.tsv', sep='\t',index_col = 'PERSONID')

In [13]:
dat_id.head()

,LABEL
PERSONID,
8ba94305557a4ec72435a9eb78001350,0
505d2a4db5dd353b775b5a30a731c02d,0
737f6cee33b47a3857b65186f5f6251e,0
3933908961f8e1780ff0b87cc5e4401d,0
c15b755463ae71dd54fa4b39d95f5170,0


In [14]:
dat.columns

Index(['APPLYNO', 'FTR0', 'FTR1', 'FTR2', 'FTR3', 'FTR4', 'FTR5', 'FTR6',
       'FTR7', 'FTR8', 'FTR9', 'FTR10', 'FTR11', 'FTR12', 'FTR13', 'FTR14',
       'FTR15', 'FTR16', 'FTR17', 'FTR18', 'FTR19', 'FTR20', 'FTR21', 'FTR22',
       'FTR23', 'FTR24', 'FTR25', 'FTR26', 'FTR27', 'FTR28', 'FTR29', 'FTR30',
       'FTR31', 'FTR32', 'FTR33', 'FTR34', 'FTR35', 'FTR36', 'FTR37', 'FTR38',
       'FTR39', 'FTR40', 'FTR41', 'FTR42', 'FTR43', 'FTR44', 'FTR45', 'FTR46',
       'FTR47', 'FTR48', 'FTR49', 'FTR50', 'FTR51', 'CREATETIME'],
      dtype='object')

In [15]:
feature_dict = {'num':['FTR0', 'FTR1', 'FTR2', 'FTR3', 'FTR4', 'FTR5',
       'FTR6', 'FTR7', 'FTR8', 'FTR9', 'FTR10', 'FTR11', 'FTR12', 'FTR13',
       'FTR14', 'FTR15', 'FTR16', 'FTR17', 'FTR18', 'FTR19', 'FTR20', 'FTR21',
       'FTR22', 'FTR23', 'FTR24', 'FTR25', 'FTR26', 'FTR27', 'FTR28', 'FTR29',
       'FTR30', 'FTR31', 'FTR32', 'FTR33', 'FTR34', 'FTR35', 'FTR36', 'FTR37',
       'FTR38', 'FTR39', 'FTR40', 'FTR41', 'FTR42', 'FTR43', 'FTR44', 'FTR45',
       'FTR46', 'FTR47', 'FTR48', 'FTR49', 'FTR50'],'cat':['APPLYNO','FTR51', 'CREATETIME']}

deal_col = {}
for col in dat.columns:
    if col in feature_dict['num']:
        deal_col[col] = [np.mean]
    else:
        deal_col[col] = [ pd.Series.nunique]
#deal_col.pop('PERSONID')

df_agg = data.groupby('PERSONID').agg(deal_col)
df_agg.head()

,APPLYNO,FTR0,FTR1,FTR2,FTR3,FTR4,FTR5,FTR6,FTR7,FTR8,...,FTR43,FTR44,FTR45,FTR46,FTR47,FTR48,FTR49,FTR50,FTR51,CREATETIME
,nunique,mean,mean,mean,mean,mean,mean,mean,mean,mean,...,mean,mean,mean,mean,mean,mean,mean,mean,nunique,nunique
PERSONID,,,,,,,,,,,,,,,,,,,,,
000e3ee6e04a8b18871e1c85ffab6e49,81,0.052956,0.0,0.0,0.0,0.0,0.000318,0.0,0.0,0.000010,...,0.000519,0.050487,0.0,0.0,0.000010,0.047401,0.0,0.000000,42,19
000eba50dc8c2e3899070c6b82986814,101,0.052685,0.0,0.0,0.0,0.0,0.000198,0.0,0.0,0.000510,...,0.000435,0.047859,0.0,0.0,0.000004,0.047860,0.0,0.000000,50,50
00139506cbd54b670e1e6dec234190e1,39,0.048932,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000717,0.048932,0.0,0.0,0.000026,0.066026,0.0,0.000000,21,26
0014fdb3625079de4a0d2131fd860641,153,0.065979,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.000179,0.063276,0.0,0.0,0.000676,0.046481,0.0,0.000000,51,62
001de94e8d0d152e26ad87cb409003f9,58,0.059370,0.0,0.0,0.0,0.0,0.000322,0.0,0.0,0.000007,...,0.000359,0.047876,0.0,0.0,0.000028,0.047876,0.0,0.000138,31,29


In [16]:
df_derive = pd.concat([df_agg,dat_id],axis = 1)

In [17]:
df_agg.shape

(17500, 54)

In [18]:
df_derive.head()

,"(APPLYNO, nunique)","(FTR0, mean)","(FTR1, mean)","(FTR2, mean)","(FTR3, mean)","(FTR4, mean)","(FTR5, mean)","(FTR6, mean)","(FTR7, mean)","(FTR8, mean)",...,"(FTR44, mean)","(FTR45, mean)","(FTR46, mean)","(FTR47, mean)","(FTR48, mean)","(FTR49, mean)","(FTR50, mean)","(FTR51, nunique)","(CREATETIME, nunique)",LABEL
000e3ee6e04a8b18871e1c85ffab6e49,81,0.052956,0.0,0.0,0.0,0.0,0.000318,0.0,0.0,0.000010,...,0.050487,0.0,0.0,0.000010,0.047401,0.0,0.000000,42,19,0.0
000eba50dc8c2e3899070c6b82986814,101,0.052685,0.0,0.0,0.0,0.0,0.000198,0.0,0.0,0.000510,...,0.047859,0.0,0.0,0.000004,0.047860,0.0,0.000000,50,50,0.0
00139506cbd54b670e1e6dec234190e1,39,0.048932,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.048932,0.0,0.0,0.000026,0.066026,0.0,0.000000,21,26,0.0
0014fdb3625079de4a0d2131fd860641,153,0.065979,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.063276,0.0,0.0,0.000676,0.046481,0.0,0.000000,51,62,NaN
001de94e8d0d152e26ad87cb409003f9,58,0.059370,0.0,0.0,0.0,0.0,0.000322,0.0,0.0,0.000007,...,0.047876,0.0,0.0,0.000028,0.047876,0.0,0.000138,31,29,0.0


In [19]:
df_derive.LABEL.fillna(2,inplace = True)


In [20]:
names = [i[0] + '_' + i[1] for i in df_derive.columns if i!= 'LABEL']
names.append('LABEL')

In [21]:
names

['APPLYNO_nunique',
 'FTR0_mean',
 'FTR1_mean',
 'FTR2_mean',
 'FTR3_mean',
 'FTR4_mean',
 'FTR5_mean',
 'FTR6_mean',
 'FTR7_mean',
 'FTR8_mean',
 'FTR9_mean',
 'FTR10_mean',
 'FTR11_mean',
 'FTR12_mean',
 'FTR13_mean',
 'FTR14_mean',
 'FTR15_mean',
 'FTR16_mean',
 'FTR17_mean',
 'FTR18_mean',
 'FTR19_mean',
 'FTR20_mean',
 'FTR21_mean',
 'FTR22_mean',
 'FTR23_mean',
 'FTR24_mean',
 'FTR25_mean',
 'FTR26_mean',
 'FTR27_mean',
 'FTR28_mean',
 'FTR29_mean',
 'FTR30_mean',
 'FTR31_mean',
 'FTR32_mean',
 'FTR33_mean',
 'FTR34_mean',
 'FTR35_mean',
 'FTR36_mean',
 'FTR37_mean',
 'FTR38_mean',
 'FTR39_mean',
 'FTR40_mean',
 'FTR41_mean',
 'FTR42_mean',
 'FTR43_mean',
 'FTR44_mean',
 'FTR45_mean',
 'FTR46_mean',
 'FTR47_mean',
 'FTR48_mean',
 'FTR49_mean',
 'FTR50_mean',
 'FTR51_nunique',
 'CREATETIME_nunique',
 'LABEL']

In [22]:
df_derive.columns = names

In [23]:
df_derive.index.set_names('PERSONID', inplace= True)

In [24]:
df_derive.to_csv('data/train_derive.csv')

In [26]:
df_derive.head(30)

,APPLYNO_nunique,FTR0_mean,FTR1_mean,FTR2_mean,FTR3_mean,FTR4_mean,FTR5_mean,FTR6_mean,FTR7_mean,FTR8_mean,...,FTR44_mean,FTR45_mean,FTR46_mean,FTR47_mean,FTR48_mean,FTR49_mean,FTR50_mean,FTR51_nunique,CREATETIME_nunique,LABEL
PERSONID,,,,,,,,,,,,,,,,,,,,,
000e3ee6e04a8b18871e1c85ffab6e49,81,0.052956,0.0,0.000000,0.0,0.000000,0.000318,0.0,0.000000,9.713944e-06,...,0.050487,0.000000e+00,0.0,0.000010,0.047401,0.000000,0.000000,42,19,0.0
000eba50dc8c2e3899070c6b82986814,101,0.052685,0.0,0.000000,0.0,0.000000,0.000198,0.0,0.000000,5.095469e-04,...,0.047859,0.000000e+00,0.0,0.000004,0.047860,0.000000,0.000000,50,50,0.0
00139506cbd54b670e1e6dec234190e1,39,0.048932,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000e+00,...,0.048932,0.000000e+00,0.0,0.000026,0.066026,0.000000,0.000000,21,26,0.0
0014fdb3625079de4a0d2131fd860641,153,0.065979,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000e+00,...,0.063276,0.000000e+00,0.0,0.000676,0.046481,0.000000,0.000000,51,62,2.0
001de94e8d0d152e26ad87cb409003f9,58,0.059370,0.0,0.000000,0.0,0.000000,0.000322,0.0,0.000000,6.506872e-06,...,0.047876,0.000000e+00,0.0,0.000028,0.047876,0.000000,0.000138,31,29,0.0
001df6145dea5c48e752d8471544969f,69,0.055077,0.0,0.000046,0.0,0.000371,0.000328,0.0,0.000372,1.852724e-05,...,0.046894,0.000000e+00,0.0,0.000036,0.046895,0.000000,0.000177,37,32,0.0
0021241a49f6aa41736ca2d88106ead6,194,0.051645,0.0,0.000000,0.0,0.000317,0.000158,0.0,0.000317,3.063632e-05,...,0.049441,0.000000e+00,0.0,0.000044,0.053364,0.000000,0.000000,70,72,0.0
0022b596fad835d63f008ec18e72f7ab,36,0.069869,0.0,0.000000,0.0,0.000000,0.000342,0.0,0.000000,0.000000e+00,...,0.046721,0.000000e+00,0.0,0.000006,0.046723,0.000000,0.000000,19,18,1.0
002d78ebb7c8b708b909e830224f59bf,25,0.066945,0.0,0.000000,0.0,0.000000,0.000187,0.0,0.000000,2.329621e-06,...,0.051948,0.000000e+00,0.0,0.000035,0.046945,0.000000,0.002334,14,13,2.0
